---

# CSCI 3202, Spring 2024
# Homework 3
# Due: Friday March 8, 2024 at 5:59 PM

<br> 

### Your name: Michael Marquez

<br> 

---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Maximizing an Objective Function with a Genetic Algorithm 

Suppose we've lost the index card with our favorite cupcake recipe. We know the ingredients of the cake, but cannot remember the exact amount of each ingredient. We decide to use a genetic algorithm to generate the  ingredient amounts. With each iteration of the genetic algorithm, we bake the cupcakes and taste-test them. We achieve our goal and stop running the genetic algorithm when we get to the actual recipe: 

* 1 tsp salt 
* 3 tsp baking powder 
* 2 cups all-purpose flour 
* 1 cup butter 
* 1 cup granulated sugar 
* 4 large eggs
* 1 tsp vanilla extract
* 1 cup buttermilk 

In [2]:
target = [1, 3, 2, 1, 1, 4, 1, 1]

An example starting state for a member of our population might look like: $state = [1, 2, 100, 36, 60, 3, 5, 50]$

### (1a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [32]:
def recipe_success(state):
    # Your code here.
    # sum of abs(difference of lists)
    val = 0
    if len(state) != len(target): # lists aren't same length
        return None
    else:
        for (s,t) in zip(state,target):
            if s != t: val += 1
        
        return len(target) - val # number we have correct

In [33]:
# Write your own test case to make sure that the target element achieves
# the fitness score you would expect it to (this will vary depending on
# what you did here.)

state = [1,2,100,36,60,3,5,50]
recipe_success(state)

1

### (1b) 

Using our in class notebook "Lecture 16 - Genetic Algorithms.ipynb" as your guide, write a genetic algorithm that starts with a population of 100 randomly generated "recipes/states/members" and uses the objective function you wrote in **(1a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from integers between 0 and 100 
- Allow for mutations in your population with an overall probability of mutation set to p = 0.2
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 50 iterations ("generations"). Do you hit your target?

In [34]:
#Your code here

# gene is random number between 0 and 100
# chromosome is list of genes which is state of Individual

import random

class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        total = sum(performance)
        p_reproduce = [perf/sum(performance) for perf in performance]
        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child):
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        child[gene] = random.randint(0,100)
        return child


In [52]:
# generate random population
def mutated_gene():
    return random.randint(0,100)

def create_chromosome():
    return [mutated_gene() for i in range(len(target))]

population = []

for x in range(200): # initial population
    
    new_lst = list(create_chromosome())
    population.append(new_lst)
    
#for i in range(len(population)):
    #print(f'Individual: {i}, State: {population[i].state}')

#a = population[0]
#b = population[1]
#abzip = list(zip(a.chromosome,b.chromosome))
#print(f'Zip: {abzip}')

#gen_prob = problem(population,recipe_success,0.2,0)

#(fit,pop) = sorted(zip(gen_prob.fitness(), gen_prob.population))
#print(fit)

In [53]:
def gen_algo(problem,n_iter):
    generation = 0 # current generation and number of iterations
    
    while generation < n_iter:

        new_generation = []
        
        for x in range(0, len(problem.population)):
            
            # reproducing using two random parents to create entirely new generation
            p_reproduce = problem.fitness()
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False) # excluding elite
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]

            child = problem.reproduce(parent1, parent2)
            
            # mutation (changes random ingredient to random number in selected recipe)
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            new_generation.append(child)
        
        # replace current population with new generation of children
        problem.population = new_generation
        
        # evaluate each recipe based on how many ingredient counts are off
        performance = [problem.objective_function(member) for member in problem.population]
        best_member = max(zip(performance,problem.population))
        
        # checking if the first item in sorted list matches the target ingredient counts
        if best_member[0] >= problem.fitness_goal:
            print(f'Goal Reached: {problem.population[best_member[0]]}')
            return best_member
        
        #print(f'Generation: {generation}\t Performances: {sorted_perf[0]}')
        print(f'Generation: {generation}\tClosest Child Chromosome: {best_member}')

        generation += 1

        
        
gen_prob = problem(population,recipe_success,0.2,len(target))
gen_algo(gen_prob,50)

Generation: 0	Closest Child Chromosome: (2, [97, 52, 1, 70, 99, 4, 3, 1])
Generation: 1	Closest Child Chromosome: (3, [29, 35, 18, 70, 1, 4, 3, 1])
Generation: 2	Closest Child Chromosome: (4, [32, 3, 2, 1, 39, 45, 1, 19])
Generation: 3	Closest Child Chromosome: (4, [96, 41, 2, 1, 37, 11, 1, 1])
Generation: 4	Closest Child Chromosome: (5, [32, 3, 2, 82, 1, 4, 3, 1])
Generation: 5	Closest Child Chromosome: (6, [1, 3, 36, 1, 1, 4, 1, 26])
Generation: 6	Closest Child Chromosome: (6, [32, 3, 2, 1, 1, 40, 1, 1])
Generation: 7	Closest Child Chromosome: (7, [1, 3, 2, 1, 1, 33, 1, 1])
Goal Reached: [96, 41, 2, 37, 52, 4, 1, 30]


(8, [1, 3, 2, 1, 1, 4, 1, 1])

### (1c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 200, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation, does that affect the number of generations it takes to achieve the goal recipe? How so?

Alternate questions to answer if Target not hit:
- Report whether you minimized of maximized the objective function and whether that led to any major changes in how you designed the probability of reproduction. A couple sentences here is fine.
 
- Report how many ingredients you ended up matching. e.g. target = [0.5, 3, 2.5, 1, 1.5, 4, 1, 1.25], perhaps your algorithm achieved [1.5, 3, 8, 1, 1, 100, 56, 1, 1.25], then you would have matched 4 of the ingredient values.
 
- Report how many iterations you tried in order to get this answer. (Don't burn up your machine in the process)

- With only 100 members in the population and 50 iterations, I did not fully meet the target. I was close with (49, [1,75,2,79,1,4,1,36]) having 5 of the ingredient counts correct.
- Changing the initial population size to 200, I reach the goal much faster in only 8 iterations.
- If I change the probability of mutation it does affect the number of generations it takes to achieve the goal. Making the probability higher, causes more random mutations which makes the algorithm take longer to finish while having no mutations means that the mutations can't interfere with the algorithm.

In [19]:
# Your code here.